# Get Around analysis

In [1]:
# Import librairies
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

In [2]:
# Import dataset
filename = "data/get_around_delay_analysis.xlsx"
df = pd.read_excel(filename)

## Overview

In [3]:
print("Display of the first lines of the dataset: ")
display(df.head())

Display of the first lines of the dataset: 


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


In [4]:
print(f"Shape of the dataframe (number of rows, number of columns) : {format(df.shape)}\n")

print("Display of the types of the columns :")
print(df.info())

Shape of the dataframe (number of rows, number of columns) : (21310, 7)

Display of the types of the columns :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  int64  
 1   car_id                                      21310 non-null  int64  
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.1+ MB
None


In [5]:
# Rename columns
df = df.rename(columns={'delay_at_checkout_in_minutes':'delay_at_checkout',
                        'time_delta_with_previous_rental_in_minutes':'time_delta_with_previous_rental'})

In [6]:
# Values taken by checkin_type and state
for var in ["checkin_type", "state"]:
    print(f"{df[var].value_counts(dropna=False)}\n")

checkin_type
mobile     17003
connect     4307
Name: count, dtype: int64

state
ended       18045
canceled     3265
Name: count, dtype: int64



In [7]:
print(f"The dataframe contains {df['rental_id'].nunique()} unique values of rental ids and {df['car_id'].nunique()} unique values of cars.")

The dataframe contains 21310 unique values of rental ids and 8143 unique values of cars.


In [8]:
print("Basic statistics: ")
display(df.describe(include = "all"))

Basic statistics: 


,rental_id,car_id,checkin_type,state,delay_at_checkout,previous_ended_rental_id,time_delta_with_previous_rental
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000


In [9]:
# Missing values
print("Number of missing values: ")
display(df.isnull().sum())
display(df.isnull().sum() / len(df) * 100)

Number of missing values: 


rental_id                              0
car_id                                 0
checkin_type                           0
state                                  0
delay_at_checkout                   4964
previous_ended_rental_id           19469
time_delta_with_previous_rental    19469
dtype: int64

rental_id                           0.000000
car_id                              0.000000
checkin_type                        0.000000
state                               0.000000
delay_at_checkout                  23.294228
previous_ended_rental_id           91.360863
time_delta_with_previous_rental    91.360863
dtype: float64

In [10]:
# Duplicates
# We check for duplicates in rental_id, in rental_id x car_id

nb_dup_rentalid = df.dropna(subset=['rental_id']).duplicated(['rental_id']).sum()
nb_dup_rentalid_carid = df.dropna(subset=['rental_id', 'car_id']).duplicated(['rental_id', 'car_id']).sum()

print(f"The dataframe contains {nb_dup_rentalid} duplicates of rental ids, and {nb_dup_rentalid_carid} duplicates of (rental id, car id).")

The dataframe contains 0 duplicates of rental ids, and 0 duplicates of (rental id, car id).


In [11]:
# Check for inconsistencies
# Look at missing values of delay at checkout
# Cancelled rentals have logically no delay at checkout, but ended rentals should have non missing values.

df_delay_na = df.loc[df['delay_at_checkout'].isnull()]
print(f"Number of missing values of delay at checkout: {len(df_delay_na)}")

print(f"\nStatus of rentals without delay at checkout(%):\n{round(df_delay_na['state'].value_counts(dropna=False), 1)} \n{round(df_delay_na['state'].value_counts(normalize=True, dropna=False)*100, 1)}")

display(df_delay_na.loc[df_delay_na['state']=="ended"].head())

Number of missing values of delay at checkout: 4964

Status of rentals without delay at checkout(%):
state
canceled    3264
ended       1700
Name: count, dtype: int64 
state
canceled    65.8
ended       34.2
Name: proportion, dtype: float64


,rental_id,car_id,checkin_type,state,delay_at_checkout,previous_ended_rental_id,time_delta_with_previous_rental
4,511440,313932,mobile,ended,NaN,NaN,NaN
23,521156,392479,mobile,ended,NaN,537298.0,0.0
29,522717,335353,mobile,ended,NaN,NaN,NaN
34,525044,349751,mobile,ended,NaN,510607.0,60.0
45,529347,391792,mobile,ended,NaN,NaN,NaN


Inconsistency: 1700 ended rentals have missing values for delay at checkout.

In [12]:
# Add delay at checkout for previous ended rental_id
df_previous = df.copy()
col_to_keep = ['rental_id', 'car_id','delay_at_checkout']
df_previous = df_previous[col_to_keep]
df_previous = df_previous.rename(columns={'rental_id':'previous_ended_rental_id', 
                                            'delay_at_checkout':'previous_delay_at_checkout'})
df_previous = df_previous.set_index(['previous_ended_rental_id','car_id'])
df = df.set_index(['previous_ended_rental_id','car_id'])
df_merged = df.join(df_previous, how='left')
df_merged = df_merged.reset_index()

df_merged.head()

,previous_ended_rental_id,car_id,rental_id,checkin_type,state,delay_at_checkout,time_delta_with_previous_rental,previous_delay_at_checkout
0,NaN,363965,505000,mobile,canceled,NaN,NaN,NaN
1,NaN,269550,507750,mobile,ended,-81.0,NaN,NaN
2,NaN,359049,508131,connect,ended,70.0,NaN,NaN
3,NaN,299063,508865,connect,canceled,NaN,NaN,NaN
4,NaN,313932,511440,mobile,ended,NaN,NaN,NaN


In [13]:
print(f"Number of observations before merge: {len(df)}")
print(f"Number of observations after merge: {len(df_merged)}")

Number of observations before merge: 21310
Number of observations after merge: 21310


In [14]:
# Add indicator of presence of a previous rental
df_merged['has_previous_rental'] = 0
df_merged.loc[df_merged['previous_ended_rental_id'].notnull(), 'has_previous_rental'] = 1
    
# Add checkout status: early or on-time vs late
df_merged['checkout_status'] = "undefined"
df_merged.loc[df_merged['delay_at_checkout']<=0, 'checkout_status'] = "early or on-time"
df_merged.loc[df_merged['delay_at_checkout']>0, 'checkout_status'] = "late"
    
df_merged['previous_checkout_status'] = "no previous rental"
df_merged.loc[df_merged['previous_delay_at_checkout']<=0, 'previous_checkout_status'] = "previous rental early or on-time"
df_merged.loc[df_merged['previous_delay_at_checkout']>0, 'previous_checkout_status'] = "previous rental late"
    
df_merged['previous_checkout_status_regroup'] = "no previous rental or previous rental early/on-time"
df_merged.loc[df_merged['previous_delay_at_checkout']>0, 'previous_checkout_status_regroup'] = "previous rental late"

In [15]:
# Add conflict indicator
df_merged['conflict'] = "undefined"
mask_late_conflict = (df_merged['previous_checkout_status']=='previous rental late') & (df_merged['previous_delay_at_checkout']>df_merged['time_delta_with_previous_rental'])
mask_late_no_conflict = (df_merged['previous_checkout_status']=='previous rental late') & (df_merged['previous_delay_at_checkout']<=df_merged['time_delta_with_previous_rental'])
mask_ontime = (df_merged['previous_checkout_status']=='previous rental early or on-time')
mask_no_previous_rental = (df_merged['previous_checkout_status']=='no previous rental')
mask_late_with_undefined_impact = (df_merged['previous_checkout_status']=='previous rental late') & ((df_merged['previous_delay_at_checkout'].isnull()| df_merged['time_delta_with_previous_rental'].isnull()))
df_merged.loc[mask_late_conflict, 'conflict'] = "conflict with the next rental"
df_merged.loc[mask_late_no_conflict, 'conflict'] = "no conflict"
df_merged.loc[mask_ontime, 'conflict'] = "no conflict"
df_merged.loc[mask_no_previous_rental, 'conflict'] = "no conflict"
df_merged.loc[mask_late_with_undefined_impact, 'conflict'] = "missing values"

# Add waiting time: previous delay at checkout - time delta with previous rental
df_merged['waiting_time'] = np.nan
df_merged.loc[(df_merged['previous_delay_at_checkout'].notnull()) & (df_merged['time_delta_with_previous_rental'].notnull()), 'waiting_time'] = df_merged['previous_delay_at_checkout'] - df_merged['time_delta_with_previous_rental']
    

In [16]:
# Nb unique cars
print(f"Nb of unique cars: {df_merged['car_id'].nunique()}")


Nb of unique cars: 8143


In [17]:
# Distribution of type of check-in

fig = px.pie(df_merged, names='checkin_type', width=500)
fig.update_traces(
    textinfo='percent+label',
)

fig.update_layout(
    title={
        'text': 'Check-in type repartition',
        'x': 0.5, 
        'xanchor': 'center',
        'yanchor': 'top'
    },
    showlegend=False
)
fig.show()

In [18]:
# Distribution of rental states

fig = px.pie(df_merged, names='state', width=500)
fig.update_traces(
    textinfo='percent+label',
)

fig.update_layout(
    title={
        'text': 'Rental state repartition',
        'x': 0.5, 
        'xanchor': 'center',
        'yanchor': 'top'
    },
    showlegend=False
)
fig.show()

In [19]:
# Repartition of state by checkin type
df_agg = df_merged.groupby(['state', 'checkin_type']).size().reset_index(name='count')
total = df_agg['count'].sum()
df_agg['proportion'] = df_agg['count'] / df_agg.groupby('checkin_type')['count'].transform('sum') * 100
display(df_agg)

# Création du graphique avec le DataFrame agrégé
fig = px.bar(df_agg,
             x='checkin_type',
             y='proportion',
             color='state',
             title='Repartition of completed vs cancelled rentals by check-in type',
             labels={'state': 'Rental state', 'type_check_in': 'Check-in type', 'counts': 'Number of rentals'},
             barmode='stack',
             text=df_agg['proportion'].apply(lambda x: f'{x:.1f}%'),
             width=700)

fig.update_layout(
    xaxis_title='Rental state',
    yaxis_title='Proportion of rentals',
    legend_title='Check-in type',
    title={'x': 0.5}
)

fig.update_traces(
    textposition='inside',
    insidetextanchor='middle'
)

fig.show()

,state,checkin_type,count,proportion
0,canceled,connect,798,18.527978
1,canceled,mobile,2467,14.509204
2,ended,connect,3509,81.472022
3,ended,mobile,14536,85.490796


In [20]:
# Delay by check-in type

df_ended_delay = df_merged.loc[(df_merged['state']=="ended") & (df_merged['delay_at_checkout'].notnull())]

df_delay_agg = df_ended_delay.groupby(['checkout_status']).size().reset_index(name='count')
df_delay_agg['proportion'] = round(df_delay_agg['count'] / df_delay_agg['count'].sum() * 100, 1)
display(df_delay_agg)

df_delay_checkin = df_ended_delay.groupby(['checkout_status', 'checkin_type']).size().reset_index(name='count')
total = df_delay_checkin['count'].sum()
df_delay_checkin['proportion'] = round(df_delay_checkin['count'] / df_delay_checkin.groupby('checkin_type')['count'].transform('sum') * 100, 1)
display(df_delay_checkin)

,checkout_status,count,proportion
0,early or on-time,6941,42.5
1,late,9404,57.5


,checkout_status,checkin_type,count,proportion
0,early or on-time,connect,1943,57.1
1,early or on-time,mobile,4998,38.6
2,late,connect,1459,42.9
3,late,mobile,7945,61.4


In [21]:
fig = px.histogram(
        df_merged, 
        x="delay_at_checkout", 
        marginal="box", 
        title=f"Distribution of delay at checkout", 
        height=400, 
        width=700)
fig.update_xaxes(range=[-360, 360])
fig.update_layout(
    xaxis_title='Delay at checkout (min)',
    yaxis_title='Nb rentals',
    title={'x': 0.5}
)
fig.update_layout(title={'x': 0.5})

fig.show()

In [22]:
for checkin in ['connect', 'mobile']:
    fig = px.histogram(
            df_merged.loc[df_merged['checkin_type']==checkin], 
            x="delay_at_checkout", 
            marginal="box", 
            title=f"Distribution of delay at checkout for {checkin} type", 
            height=400, 
            width=700)
    fig.update_xaxes(range=[-360, 360])
    fig.update_layout(
        xaxis_title='Delay at checkout (min)',
        yaxis_title='Nb rentals',
        title={'x': 0.5}
    )
    fig.update_layout(title={'x': 0.5})

    fig.show()

In [24]:
fig = px.histogram(
        df, 
        x="time_delta_with_previous_rental", 
        marginal="box", 
        title=f"Distribution of time delta with previous rental", 
        height=400, 
        width=700)
fig.update_layout(
    xaxis_title='Time delta with previous rental (min)',
    yaxis_title='Count of rentals',
    title={'x': 0.5}
)
fig.show()

The dataset covers a fleet of 8143 cars.  
21310 rentals are registered.  
80% rentals use Mobile check-in, only 20% use Connect check-in.  
An overall 15% cancellation rate: the cancellation rate is higher for Connect (18.5%) than Mobile (14.5%).

58% of rentals experience delays.  
Delay at checkout has extreme values (negative as positive) so we display the median instead of the mean.
The median delay at checkout is 9 min, meaning half of the rentals (with non missing delay at checkout) return the car with 9-min delay.
1st quartile is -36 min, meaning 25% of the rentals (with non missing delay at checkout) return the car early (36 min in advance).  

Mobile users tend to be more late are checkout than Connect ones:  
- A higher proportion of mobile users are late (61%) compared to Connect users (43%).
- Half of all mobile users are delayed by 14 minutes or more, whereas half of all Connect users are at least 9 minutes early for checkout.


# Impact of a late checkout on the following rental

In this section we focus on the subset of consecutive rentals.  
How often are drivers late for the next check-in? How does it impact the next driver?  

In [25]:
# Create the subset: rentals with a preceding one
df_consecutive = df_merged.loc[(df_merged['has_previous_rental']==1) & (df_merged['previous_delay_at_checkout'].notnull())].copy()
print(f"{len(df_consecutive)} rentals have a preceding one, accounting for {round(len(df_consecutive)/len(df_merged)*100, 1)}% of total rentals")

1729 rentals have a preceding one, accounting for 8.1% of total rentals


In [26]:
df_consecutive_agg = df_consecutive.groupby(["previous_checkout_status"], dropna=False).agg(
    count=("rental_id", "count"),
    median_delay_at_checkout=("previous_delay_at_checkout", "median")).reset_index()
df_consecutive_agg['proportion'] = df_consecutive_agg['count']/df_consecutive_agg['count'].sum()*100
display(df_consecutive_agg.round(1))

,previous_checkout_status,count,median_delay_at_checkout,proportion
0,previous rental early or on-time,856,-55.0,49.5
1,previous rental late,873,42.0,50.5


In [27]:
fig = px.pie(df_consecutive_agg, 
                names='previous_checkout_status',
                values='count',
                color='previous_checkout_status',  
                height=420)
fig.update_traces(
    textinfo='percent+label',
    texttemplate='%{label}<br>%{percent:.0%}',  
    insidetextorientation='horizontal',
    textposition='inside'
    )
fig.update_layout(
    title={
        'text': 'Repartition of previous checkout delay status',
        'x': 0.5, 
        'xanchor': 'center',
        'y': 0.95,
        'yanchor': 'top'
    },
    showlegend=False,
    margin=dict(l=20, r=20, t=80, b=20)
)
fig.show()

In [28]:
for checkin in ['mobile', 'connect']:
    print(f"Check-in type : {checkin}")
    df_temp = df_consecutive.loc[df_consecutive['checkin_type']==checkin].groupby(["previous_checkout_status"]).agg(
        count=("rental_id", "count"),
        median_delay_at_checkout=("previous_delay_at_checkout", "median"))
    df_temp['proportion'] = df_temp['count']/df_temp['count'].sum()*100
    display(df_temp.round(1))

Check-in type : mobile


,count,median_delay_at_checkout,proportion
previous_checkout_status,,,
previous rental early or on-time,369,-48.0,39.3
previous rental late,569,46.0,60.7


Check-in type : connect


,count,median_delay_at_checkout,proportion
previous_checkout_status,,,
previous rental early or on-time,487,-61.0,61.6
previous rental late,304,35.5,38.4


Consecutive rentals represents a small proportion (8.1%) of all rentals.
In half of these cases, the previous driver returns the car late, with a median delay of 42 minutes.
The proportion of late users at checkout is higher for Mobile users than for Connect users (61% vs 38%). The median delay is also higher for Mobile than for Connect: 46min for Mobile and 36 min for Connect.

In [29]:
# Study conflictual situations: when the previous driver's return delay exceeds the scheduled interval between rentals

mask_conflict = (df_merged['previous_delay_at_checkout'].notnull()) & (df_merged['time_delta_with_previous_rental'].notnull()) & (df_merged['previous_delay_at_checkout']>df_merged['time_delta_with_previous_rental'])
df_conflict = df_merged.loc[mask_conflict]

nb_conflict = len(df_conflict)
prct_conflict = round(nb_conflict/len(df_merged)*100, 1)

print(f"There are {nb_conflict} conflictual cases, accounting for {prct_conflict}% of all rentals.")

median_waiting_time = df_conflict['waiting_time'].median()
print(f"Median waiting time: {median_waiting_time} min")

df_conflict_agg = df_merged.groupby(['conflict', 'state'], dropna=False).size().reset_index(name='count')
df_conflict_agg['proportion'] = df_conflict_agg['count'] / df_conflict_agg.groupby('conflict')['count'].transform('sum') * 100
display(df_conflict_agg.round(1))


There are 218 conflictual cases, accounting for 1.0% of all rentals.
Median waiting time: 26.5 min


,conflict,state,count,proportion
0,conflict with the next rental,canceled,37,17.0
1,conflict with the next rental,ended,181,83.0
2,no conflict,canceled,3228,15.3
3,no conflict,ended,17864,84.7


In [30]:
fig = px.bar(df_conflict_agg,
            x='conflict',
            y='proportion',
            color='state',
            title='Canceled vs completed rentals by conflict status',
            barmode='stack',
            text=df_conflict_agg.apply(lambda row: f"{row['state']}: {row['proportion']:.0f}%", axis=1),
            hover_data={'count': True},
            height=450,
            width=500
            )

fig.update_layout(
    xaxis_title='Conflict status',
    yaxis_title='Proportion of rentals',
    showlegend=False,
    title={'x': 0.5,'xanchor': 'center',
        'y': 0.95,
        'yanchor': 'top'},
    margin=dict(l=20, r=20, t=50, b=20)
)
fig.update_traces(
    hovertemplate="<b>%{x}</b><br>State: %{fullData.name}<br>Proportion: %{y:.0f}%<br>Count: %{customdata[0]}<extra></extra>",
    textposition='inside',
    insidetextanchor='middle'
)
fig.show()

In [31]:
df_consecutive_late = df_consecutive.loc[df_consecutive['previous_checkout_status']=="previous rental late"].copy()
delay_median_consecutive_late = df_consecutive_late['previous_delay_at_checkout'].median()
fig = px.histogram(
    df_consecutive_late, 
    x="previous_delay_at_checkout", 
    marginal="box",
    title=f"Distribution of checkout delays for late rentals that precede another rental"
)
fig.add_vline(
    x=delay_median_consecutive_late,
    line_dash="dash",
    line_color="red",
)
fig.add_annotation(
    x=delay_median_consecutive_late,
    y=1.08, 
    xref="x", 
    yref="paper",
    text="Median delay",
    showarrow=False,
    xanchor="left",
    yanchor="top",
    font=dict(color="red")
)
fig.update_traces(xbins=dict(start=-360, end=360, size=30), selector=dict(type="histogram"))    
fig.update_xaxes(range=[0, 360])
fig.update_layout(
    xaxis_title='Delay at checkout (min)',
    yaxis_title='Number of rentals',
    title={'x': 0.5,'xanchor': 'center',
        'y': 0.95,
        'yanchor': 'top'},
    margin=dict(b=70),
    xaxis=dict(tick0=0,dtick=60, range=[0, 360])
)
fig.show()

In [32]:
fig = px.histogram(
    df_conflict, 
    x="waiting_time", 
    marginal="box",
    title=f"Distribution of waiting time in case of conflict", 
)
fig.add_vline(
    x=median_waiting_time,
    line_dash="dash",
    line_color="red",
)
fig.add_annotation(
    x=median_waiting_time,
    y=1.08, 
    xref="x", 
    yref="paper",
    text="Median delay",
    showarrow=False,
    xanchor="left",
    yanchor="top",
    font=dict(color="red")
)
fig.update_traces(xbins=dict(start=0, end=360, size=30), selector=dict(type="histogram"))
fig.update_xaxes(range=[0, 360])
fig.update_layout(
    xaxis_title='Waiting time (min)',
    yaxis_title='Number of rentals',
    title={'x': 0.5,'xanchor': 'center',
        'y': 0.95,
        'yanchor': 'top'},
    margin=dict(b=70),
    xaxis=dict(tick0=0,dtick=60, range=[0, 360])
)
fig.add_annotation(
    text="Bin size: 30 minutes",
    x=0.5, xref="paper",
    y=-0.26, yref="paper",
    showarrow=False,
    font=dict(size=12, color="gray"),
    align="center"
)
fig.show()

A late return does not necessarily have a negative impact on the next rental.
Conflicts arise only when the previous driver's return delay exceeds the scheduled interval between rentals. 

Overall, 218 such conflicts have been recorded, accounting for 1% of all rentals, which is relatively small.

These conflicts are problematic, as they force the next driver to wait if they choose not to cancel their reservation. 
In such cases, the median waiting time is 27 minutes.  

The cancellation rate remains comparable (~15-17%), regardless of a conflict occurrence.
Indeed, rentals may be cancelled for various reasons beyond time conflicts with the previous rental.

# Scenario analysis: buffer time between rentals

In [34]:
df_ended = df_merged.loc[(df_merged['state']=='ended')]
nb_ended = len(df_ended)

#mask_conflict1 = (df['previous_delay_at_checkout'].notnull()) & (df['time_delta_with_previous_rental'].notnull()) & (df['previous_delay_at_checkout']>df['time_delta_with_previous_rental'])
#nb_conflict = len(df.loc[mask_conflict1])  # total number of conflicts

nb_canceled = len(df_merged.loc[df_merged['state']=='canceled'])

# The feature has an impact on consecutive rentals (rentals preceded by another rental)
# Filter on consecutive rentals with non-missing previous delay at checkout
#df_previous = df_filter.loc[(df_filter['has_previous_rental']==1) & (df_filter['previous_delay_at_checkout'].notnull())]
nb_total = len(df_consecutive)

# Conflictual situations
#mask_conflict = (df_merged['previous_delay_at_checkout'].notnull()) & (df_merged['time_delta_with_previous_rental'].notnull()) & (df_merged['previous_delay_at_checkout']>df_filter['time_delta_with_previous_rental'])
#df_conflict = df_merged.loc[mask_conflict]

# Canceled rentals which are preceded by another rental
df_canceled_hasprevious = df_merged.loc[(df_merged['state']=='canceled') & (df_merged['has_previous_rental']==1)]

# Ended rentals which are preceded by another rental
df_ended_hasprevious = df_merged.loc[(df_merged['state']=='ended') & (df_merged['has_previous_rental']==1)]
nb_ended_previous = len(df_ended_hasprevious)


In [35]:
threshold_range = sorted(df_merged['time_delta_with_previous_rental'].dropna().unique())

results = []

delta = df_ended['time_delta_with_previous_rental']
previous_delay = df_ended['previous_delay_at_checkout']

for threshold in threshold_range:
    
    count_blocked = ((df_consecutive['time_delta_with_previous_rental'].notnull()) & (df_consecutive['time_delta_with_previous_rental']<threshold)).sum()
    count_solved = (df_conflict['time_delta_with_previous_rental']<threshold).sum()
    count_blocked_ended = (df_ended_hasprevious['time_delta_with_previous_rental']<threshold).sum()
    count_blocked_canceled = (df_canceled_hasprevious['time_delta_with_previous_rental']<threshold).sum()

    prct_blocked = count_blocked/len(df_merged)*100
    prct_solved = count_solved/nb_conflict*100
    prct_blocked_canceled = count_blocked_canceled/nb_canceled*100
    prct_revenue_loss = count_blocked_ended/nb_ended*100
    
    results.append({
        'threshold': threshold,
        'nb_blocked': count_blocked,
        'nb_solved': count_solved,
        'nb_blocked_canceled': count_blocked_canceled,
        'nb_blocked_ended': count_blocked_ended,
        'prct_blocked': prct_blocked,
        'prct_solved':prct_solved,
        'prct_blocked_canceled':prct_blocked_canceled,
        'prct_revenue_loss':prct_revenue_loss
    })

df_results = pd.DataFrame(results)

In [39]:
x = df_results['threshold']
y1 = df_results['nb_blocked']
y2 = df_results['nb_solved']
y3 = df_results['nb_blocked_canceled']
prct1 = df_results['prct_blocked']
prct2 = df_results['prct_solved']
prct3 = df_results['prct_blocked_canceled']

hover_prct1 = prct1.values.reshape(-1, 1)
hover_prct2 = prct2.values.reshape(-1, 1)
hover_prct3 = prct3.values.reshape(-1, 1)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=x,
        y=y1,
        mode='lines+markers',
        name='Rental offers hidden from search results',
        line=dict(color='#D52B1E'),
        customdata=hover_prct1,
        hovertemplate=(
            "Nb rental offers hidden: %{y:,}<br>" +
            "% rental offers hidden: %{customdata[0]:.1f}%<extra></extra><br>" 
        )
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=y2,
        mode='lines+markers',
        name='Conflicts solved',
        line=dict(color='#00997C'),
        customdata=hover_prct2,
        hovertemplate=(
            "Nb conflicts solved: %{y:,}<br>"
            +
            "% conflicts solved: %{customdata[0]:.1f}%<extra></extra><br>"
        )
    )
)


fig.add_trace(
    go.Scatter(
        x=x,
        y=y3,
        mode='lines+markers',
        name='Cancellations avoided',
        line=dict(color='#C5C5C5'),
        customdata=hover_prct3,
        hovertemplate=(
            "Nb cancellations avoided: %{y:,}<br>"
            +
            "% cancellations avoided: %{customdata[0]:.1f}%<extra></extra>"
        )
    )
)

fig.update_layout(
    title={
        'text': "Impact of a minimum delay threshold",
        'x': 0.5,
        'xanchor': 'center',
        'y': 0.95,
        'yanchor': 'top'
    },
    xaxis_title='Minimum delay threshold (min)',
    yaxis_title='Number of cases',
    margin=dict(b=90),
    hovermode='x unified',
    height=550,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5
)
)

fig.show()

In [38]:
# Revenue loss
fig = px.line(
    df_results,
    x="threshold",
    y="prct_revenue_loss",
    labels={"threshold": "Minimum delay threshold", "prct_revenue_loss": "%"},
    markers=True
)

fig.update_traces(
    hovertemplate="Threshold: %{x} min<br>% revenue lost: %{y:.1f}%<extra></extra>"
)

fig.update_layout(
    title={
        'text': "Potential revenue loss by threshold",
        'x': 0.5,
        'xanchor': 'center',
        'y': 0.95,
        'yanchor': 'top'
    },
    xaxis_title='Minimum delay threshold (min)',
    yaxis_title='%',
    margin=dict(b=90),
    height=550,
    showlegend=False
)

fig.show()